In [1]:
import numpy as np
import re
import collections

In [2]:
#一元n-gram 创建字典
dic = []
with open('input.txt','r') as f:
    for line in f.readlines():
        word2 = re.findall('[a-zA-Z\']+', line)
        dic.extend([word.lower() for word in word2])

In [3]:
len(set(dic))#12631个单词

12631

In [4]:
#2元模型 n-gram 创建2个单词的组合
dic_2 = []
for i in range(len(dic)-1):
    two_words = dic[i]+' '+dic[i+1]
    dic_2.append(two_words)

In [5]:
#3元 n-gram 创建3个单词的组合
dic_3 = []
for i in range(len(dic)-2):
    three_words = dic[i]+' '+dic[i+1]+' '+dic[i+2]
    dic_3.append(three_words)

In [6]:
#条件概率
dic_num = collections.Counter(dic)
dic_num_2 = collections.Counter(dic_2)
dic_num_3 = collections.Counter(dic_3)
#it's a 的概率
#p(are|we) = p(we are)/p(we)
print((dic_num_2['we are']/len(dic_num_2))/dic_num['we']/len(dic_num))

6.31277399919021e-11


In [7]:
sentence = ['i','will','tell','you']

#创建一个共现矩阵
def create_matrix(sentence):
    matrix = np.zeros([len(sentence),len(sentence)])
    for i in range(len(sentence)):
        for j in range(len(sentence)):
            matrix[i][j] = dic_num_2[sentence[i]+' '+sentence[j]]
    return matrix

#创建最大似然（MLP）的计数概率矩阵
def score_matrix(sentence,co_matrix):
    matrix = np.zeros([len(sentence),len(sentence)])
    for i in range(len(co_matrix[0])):
        for j in range(len(co_matrix[1])):
            matrix[i][j] = co_matrix[i][j]/dic_num[sentence[i]]
    return matrix

In [8]:
co_matrix = create_matrix(sentence)

In [9]:
MLP_matrix = score_matrix(sentence,co_matrix)

In [10]:
#Add-one 平滑
#先对co_matrix所有的数值加一
def add_one_op(co_matrix):
    for i in range(len(co_matrix[0])):
        for j in range(len(co_matrix[1])):
            co_matrix[i][j] = co_matrix[i][j]+1
    return co_matrix

def new_score_matrix(sentence,co_matrix):
    matrix = np.zeros([len(sentence),len(sentence)])
    for i in range(len(co_matrix[0])):
        for j in range(len(co_matrix[1])):
            matrix[i][j] = co_matrix[i][j]/(dic_num[sentence[i]]+12631)#V = 12631
    return matrix

In [11]:
new_score_matrix(sentence,add_one_op(co_matrix))

array([[2.90816030e-04, 1.64601873e-02, 1.45408015e-03, 5.81632060e-05],
       [4.16849495e-03, 7.31314904e-05, 6.58183414e-04, 3.36404856e-03],
       [2.32684402e-04, 7.75614675e-05, 7.75614675e-05, 2.63708989e-03],
       [1.52100894e-03, 4.11939920e-03, 8.87255213e-04, 8.23879840e-04]])